In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import glob

In [2]:
tf.test.is_built_with_cuda()

True

In [3]:
tf.config.list_physical_devices('GPU')

2021-10-22 09:59:39.882198: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-22 09:59:39.905959: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-22 09:59:39.906111: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
from tensorflow.keras.layers import LSTM,Concatenate, Dense, Flatten ,Activation ,Input , BatchNormalization,Dropout , Bidirectional
from tensorflow.keras.models import Sequential, Model

In [5]:
# function needed for angle wrapping
def wrap_angles(a):
    ad = np.remainder(a,2*np.pi)
    ad[ad > np.pi] -= 2*np.pi
    ad[ad < -np.pi] += 2*np.pi
    return ad

In [6]:
building1_filenames = set(glob.glob("datasets/building1/*/*.feather"))
building2_filenames = set(glob.glob("datasets/building2/*/*.feather"))
building3_filenames = set(glob.glob("datasets/building3/*/*.feather"))
training_filenames = set(glob.glob("datasets/building1/train/*.feather"))
test1_filenames = building1_filenames - training_filenames

In [7]:
import random
seed=10
random.seed(seed)
np.random.seed(seed)

In [8]:
# generate training samples

dfs = []
for file in training_filenames:
    df = pd.read_feather(file)
    dfs.append(df)


training_data = pd.concat(dfs, ignore_index=True)

training_data.head()

ValueError: No objects to concatenate

In [ ]:
checkpoints_path = os.path.join(os.getcwd(),'datasets/CheckPoints_Model_OrientNet')

if not os.path.exists(checkpoints_path):
  os.mkdir(checkpoints_path)
check_point_template_path = os.path.join(checkpoints_path,'ckpt_epoch_{epoch:03d}_loss_{loss:.4f}_vloss_{val_loss:.4f}.hdf5')
check_point_callback = tf.keras.callbacks.ModelCheckpoint(check_point_template_path)

import re #regular expresion
def get_all_checkpoints(checkpoints_path,checkpoint_main_name = 'ckpt'):
  all_checkpoints = [j for j in os.listdir(checkpoints_path) if j.startswith(checkpoint_main_name)]
  return all_checkpoints

def check_if_available_checkpoints(checkpoints_path,checkpoint_main_name = 'ckpt'):
  all_checkpoints = get_all_checkpoints(checkpoints_path,checkpoint_main_name)
  if(len(all_checkpoints) > 0):#checkpoints avilable
    all_checkpoints.sort(reverse=True)    
    latest_check_point = all_checkpoints[0]
    initial_epoch = int(re.search('epoch_(.*?)_', latest_check_point).group(1))    
  else:
    latest_check_point = None
    initial_epoch = 0
    
  return initial_epoch , latest_check_point


# Check if there are any check points initially
initial_epoch , latest_check_point = check_if_available_checkpoints(checkpoints_path)

In [ ]:
orient_learning_rate = 0.0005 # convergence  within  20  epochs \ref{IDOL}

In [ ]:
# Loss Function
class OreintLoss(tf.keras.losses.Loss):
  def call(self, y_true, y_pred):
    # custom loss L_orient = 0.5*(q [-] \hat{q})^T *Sig^{-1}* 
    W = 25
    ws = tf.constant([W,W,1,1,1,1],dtype=tf.float32)
    return tf.sqrt(tf.reduce_mean(ws*(y_pred - y_true)**2, axis=-1))

In [ ]:
class OreintNet():
    def build_model(self):
        # Building Network
        #1. Define inputs
        Acc_input = Input(shape=(100,3) , name = 'Acc_input')    
        Gyro_input = Input(shape=(100,3) , name = 'Gyro_input')
        Mag_input = Input(shape=(100,3) , name = 'Mag_input')
        
        MergedLayer = Concatenate()([Acc_input , Gyro_input, Mag_input])
        
        #2. LSTM
        LSTM1 = (LSTM(100, return_sequences=True))(MergedLayer)
        LSTM2 = (LSTM(100))(LSTM1)
        
        #3. Fully-Connected (Sigma)
        Dense1 = Dense(units=100, activation='tanh')(LSTM2)
        Dense2 = Dense(units=16, activation='tanh')(Dense1)
        sigma_output = Dense(units=6, activation='linear')(Dense2) # Sigma Outputs
        
        #4. Fully-Connected (Theta)
        Dense3 = Dense(units=100, activation='tanh')(LSTM2)
        Dense4 = Dense(units=16, activation='tanh')(Dense3)
        theta_output = Dense(units=6, activation='linear')(Dense4) # Theta Outputs
        
        #5. Define and compile The model
        Network = Model([Acc_input,Gyro_input, Mag_input] ,  [theta_output , sigma_output])
        Network.compile(loss=OreintLoss(), optimizer=tf.keras.optimizers.Adam(learning_rate=orient_learning_rate) ,metrics=['rmse'])
        return Network
        

In [ ]:
network_builder = OreintNet()
orientation_network = network_builder.build_model()

In [ ]:
orientation_network.summary()

In [ ]:
history = orientation_network.fit(
    batch_size=64
)

In [ ]:
pos_learning_rate = 0.001